# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [4]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [29]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head()

CustomerID FirstName   LastName  SalesID  ProductID           ProductName  \
0       61288      Rosa   Andersen   134196        229  Bread - Hot Dog Buns   
1       77352     Myron     Murray  6167892        229  Bread - Hot Dog Buns   
2       40094     Susan  Stevenson  5970885        229  Bread - Hot Dog Buns   
3       23548    Tricia    Vincent  6426954        229  Bread - Hot Dog Buns   
4       78981     Scott      Burch   819094        229  Bread - Hot Dog Buns   

   Quantity  
0        16  
1        20  
2        11  
3         6  
4        20

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [30]:
df = data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()
df.head()

CustomerID                    ProductName  Quantity
0          33               Apricots - Dried         1
1          33              Assorted Desserts         1
2          33        Bandage - Flexible Neon         1
3          33  Bar Mix - Pina Colada, 355 Ml         1
4          33         Beans - Kidney, Canned         1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [31]:
ratings = pd.pivot_table(df, index='ProductName',  values='Quantity', columns='CustomerID', aggfunc='sum', fill_value=0)
ratings.head()

CustomerID                         33     200    264    356    412    464    \
ProductName                                                                   
Anchovy Paste - 56 G Tube              0      0      0      0      0      0   
Appetizer - Mini Egg Roll, Shrimp      0      0      0      0      0      0   
Appetizer - Mushroom Tart              0      0      0      0      0      0   
Appetizer - Sausage Rolls              0      0      0      0      0      0   
Apricots - Dried                       1      0      0      0      1      0   

CustomerID                         477    639    649    669    ...    97697  \
ProductName                                                    ...            
Anchovy Paste - 56 G Tube              0      1      0      0  ...        0   
Appetizer - Mini Egg Roll, Shrimp      0      0      0      0  ...       25   
Appetizer - Mushroom Tart              0      1      0      0  ...       25   
Appetizer - Sausage Rolls              0      0      0      0  ...        0   
Apricots - Dried                       0      0      0      0  ...        0   

CustomerID                         97753  97769  97793  97900  97928  98069  \
ProductName                                                                   
Anchovy Paste - 56 G Tube             25      0      0      0      0      0   
Appetizer - Mini Egg Roll, Shrimp     25      0      0      0      0      0   
Appetizer - Mushroom Tart              0      0      0      0      0      0   
Appetizer - Sausage Rolls              0      0      0     25     25     25   
Apricots - Dried                      25      0      0      0      0      0   

CustomerID                         98159  98185  98200  
ProductName                                             
Anchovy Paste - 56 G Tube              0      0      0  
Appetizer - Mini Egg Roll, Shrimp      0      0      0  
Appetizer - Mushroom Tart              0     25      0  
Appetizer - Sausage Rolls              0     25      0  
Apricots - Dried                       0      0      0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [32]:
squareform(pdist(ratings.T, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [37]:
distances = pd.DataFrame(1/(1 + squareform(pdist(ratings.T, 'euclidean'))), 
                         index=ratings.columns, columns=ratings.columns)

distances.head()

CustomerID     33        200       264       356       412       464    \
CustomerID                                                               
33          1.000000  0.077421  0.087047  0.081800  0.080634  0.082709   
200         0.077421  1.000000  0.078448  0.076435  0.073693  0.075255   
264         0.087047  0.078448  1.000000  0.080070  0.081800  0.080350   
356         0.081800  0.076435  0.080070  1.000000  0.076435  0.078187   
412         0.080634  0.073693  0.081800  0.076435  1.000000  0.078711   

CustomerID     477       639       649       669      ...        97697  \
CustomerID                                            ...                
33          0.074573  0.083020  0.081503  0.080070    ...     0.004811   
200         0.075956  0.076435  0.077674  0.076923    ...     0.004824   
264         0.076923  0.080634  0.082100  0.078448    ...     0.004822   
356         0.075025  0.082403  0.077171  0.075956    ...     0.004816   
412         0.075025  0.082403  0.078187  0.078448    ...     0.004810   

CustomerID     97753     97769     97793     97900     97928     98069  \
CustomerID                                                               
33          0.004669  0.004412  0.005019  0.004312  0.004515  0.004583   
200         0.004681  0.004431  0.005047  0.004311  0.004521  0.004614   
264         0.004674  0.004416  0.005035  0.004322  0.004543  0.004595   
356         0.004671  0.004416  0.005038  0.004310  0.004526  0.004578   
412         0.004702  0.004414  0.005034  0.004318  0.004530  0.004578   

CustomerID     98159     98185     98200  
CustomerID                                
33          0.004355  0.004167  0.004333  
200         0.004367  0.004166  0.004335  
264         0.004365  0.004179  0.004333  
356         0.004365  0.004175  0.004339  
412         0.004367  0.004177  0.004349  

[5 rows x 1000 columns]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [60]:
distances_temp = distances.reset_index()
top5_customers = distances_temp.CustomerID.head(5)
top5_customers

0     33
1    200
2    264
3    356
4    412
Name: CustomerID, dtype: int64

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [65]:
data_top = data[data.CustomerID.isin(top5_customers)]
data_top.CustomerID.value_counts()

200    77
412    73
356    67
264    65
33     63
Name: CustomerID, dtype: int64

In [85]:
top_products =  data_top.ProductID.value_counts()
top_products.head()

245    5
237    4
88     4
83     4
77     4
Name: ProductID, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [92]:
df_producs = data[~data.ProductID.isin(top_products)]
df_producs.ProductID.value_counts().head()

29     186
222    182
123    181
396    181
273    180
Name: ProductID, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.